In [ ]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image, ImageDraw
import plotly.express as px
import shapely
from shapely import Point, Polygon
import supervision as sv
import torch
from tqdm.notebook import tqdm
from ultralytics import YOLO

In [ ]:
HOME = os.getcwd()
print(f"{HOME = }")
print(f"{torch.cuda.is_available() = }")

In [ ]:
model = YOLO('yolov8n.pt')  # Load a pretrained YOLO model https://docs.ultralytics.com/usage/python/

In [ ]:
# video_path = r'C:\Repos\inz\src_vids\test_lagrange_23.mkv'
# target_path = r'C:\Repos\inz\runs\test\test_polygon_zone_lagrange_23.mkv'
video_path = r'C:\Repos\inz\src_vids\test_hway_22.mkv'
target_path = r'C:\Repos\inz\runs\test\test_polygon_zone_hway_22.mkv'

In [ ]:
classes = {1:'bicycle', 2:'car', 3:'motorcycle', 5:'bus', 7:'truck'}

In [ ]:
video_info = sv.VideoInfo.from_video_path(video_path)
video_info

# Draw polygon zones
1. Choose polygon corners. Add point: CTRL + LMB. Remove last point: CTRL + RMB
2. Save polygon zone (CTRL + MMB)
3. Choose zone annotation location (ALT + LMB)
4. Save zone annotation location (ALT + MMB)
5. For creating new polygon zone repeat steps 1-4
6. When done, press ESCAPE

In [ ]:
def get_color(cmap="Pastel1", n_colors=9):
    """Color generator. Loops colors if end is reached.

    Args:

        cmap (str, optional): Cmap name. cmap from matplotlib.pyplot.get_cmap(cmap, n_colors). Defaults to 'Pastel1'.

        n_colors (int, optional): Number of colors. n_colors from matplotlib.pyplot.get_cmap(cmap, n_colors). Defaults to 9.

    Yields:

        list: List of r, g, b values. Eg.: (255, 255, 255)
    """
    cmap = plt.get_cmap(cmap, n_colors)
    all_colors = []
    for i in range(n_colors):
        all_colors.append([c * 255 for c in cmap(0 + (i / n_colors))])
    n = 0
    while True:
        yield all_colors[n % n_colors][:-1]
        n += 1


# Parameter initialization
cap = cv2.VideoCapture(video_path)
status, first_frame = cap.read()
cap.release()
good_image = first_frame.copy()
color_gen = get_color()
color = next(color_gen)
zones = dict()  # dictionary with Polygon coords
zone_annotations = dict()  # zone annotations locations
zone_colors = dict()
temp_annotation_location = None
curr_zone_id = 0

# Zone annotations text settings
tfontface = cv2.FONT_HERSHEY_SIMPLEX
tscale = 0.7
tcolor = color
tthickness = 2
tlinetype = cv2.LINE_AA


def draw_zones(event, x, y, flags, param):
    global zones, curr_zone_id, good_image, first_frame, color, tcolor, temp_annotation_location
    # Add next point to current polygon zone
    if flags == cv2.EVENT_FLAG_CTRLKEY + cv2.EVENT_FLAG_LBUTTON:
        first_frame = good_image.copy()
        if curr_zone_id in zones:
            zones[curr_zone_id].append((x, y))
        else:
            zones[curr_zone_id] = [(x, y)]
        try:
            cv2.polylines(first_frame, np.array([zones[curr_zone_id]]), True, color, 4)
        except Exception as e:
            print(e)
    # Delete last added point from current polygon zone
    elif flags == cv2.EVENT_FLAG_CTRLKEY + cv2.EVENT_FLAG_RBUTTON:
        first_frame = good_image.copy()
        try:
            zones[curr_zone_id].pop(-1)
        except IndexError:
            zones.pop(curr_zone_id, None)
        except KeyError:
            pass
        try:
            cv2.polylines(first_frame, np.array([zones[curr_zone_id]]), True, color, 4)
        except Exception as e:
            print(e)
    
    # Pick annotation locations
    elif flags == cv2.EVENT_FLAG_ALTKEY + cv2.EVENT_FLAG_LBUTTON:
        first_frame = good_image.copy()
        text = f"z{curr_zone_id-1} Now: 10 Total: 1000"
        (text_width, text_height), baseline = cv2.getTextSize(text, tfontface, tscale, tthickness)
        temp_annotation_location = (x, y)
        cv2.rectangle(first_frame, (x, y), (x + text_width, y + text_height), (0,0,0), -1)
        cv2.rectangle(first_frame, (x, y), (x + text_width, y + text_height), (0,0,0), 20)
        cv2.putText(first_frame, text, (x, y+text_height), tfontface, tscale, tcolor, tthickness, tlinetype)
        
    # Save annotation location
    elif flags == cv2.EVENT_FLAG_ALTKEY + cv2.EVENT_FLAG_MBUTTON:
        good_image = first_frame
        zone_annotations[curr_zone_id-1] = temp_annotation_location

    # Save current polygon zone and move to next one
    elif flags == cv2.EVENT_FLAG_CTRLKEY + cv2.EVENT_FLAG_MBUTTON:
        good_image = first_frame
        zone_colors[curr_zone_id] = color
        curr_zone_id += 1
        tcolor = color
        color = next(color_gen)
    cv2.imshow("drawing polygon zones", first_frame)


cv2.namedWindow("drawing polygon zones")
cv2.setMouseCallback("drawing polygon zones", draw_zones)
while True:
    cv2.imshow("drawing polygon zones", first_frame)
    if cv2.waitKey(10) == 27:
        break
cv2.destroyAllWindows()

In [ ]:
zones

In [ ]:
# wsp okienkowe inne niz matematyczne, ale to nie ma znaczenia dla mojego zastosowania
Polygon([[xy[0], -xy[1]] for xy in zones[0]])
# Polygon([[xy[0], xy[1]] for xy in zones[0]])

In [ ]:
# Process Whole Video
results = model.track(source=video_path,
                      stream=True,  # stream=True should be used for long videos to avoid OOM errors
                      tracker='botsort.yaml',  # custombotsort.yaml
                      persist=True,
                      classes=list(classes.keys()),
                      imgsz=640,
                      conf=0.25,
                      iou=0.7,
                      project=r'C:\Repos\inz\runs',
                      name='test_polygon_zone_lagrange_23',
                      save=True,
                      show=False,
                      line_width=3,
                      # save_txt=True,
                      # save_conf=True,
                      # agnostic_nms=False,
                      # visualize=False,
                      # save_crop=True,
                      )

In [ ]:
# Choose point location for polygon contains checks 
CHECK_MODE = 'CENTER'
CHECK_MODE = 'BOTTOM_CENTER'

# Process Whole Video and create DataFrame
video_info = sv.VideoInfo.from_video_path(video_path)
with sv.VideoSink(target_path, video_info) as s:
    temp_table = []  # list with data for df creation
    zones_objs_sets = {k:set() for k in zones.keys()}  # for each zone: unique IDs of vehicles which crossed a zone 

    for frame, r in enumerate(results):
        # detections = sv.Detections.from_yolov8(r)
        detections = sv.Detections(xyxy=r.boxes.xyxy.cpu().numpy(),
                                    confidence=r.boxes.conf.cpu().numpy(),
                                    class_id=r.boxes.cls.cpu().numpy().astype(int),
                                    tracker_id=r.boxes.id.cpu().numpy().astype(int))

        # bboxes graphics settings
        box_annotator = sv.BoxAnnotator(color=sv.ColorPalette.default(),  # sv.ColorPalette.default()
                                        thickness=2,
                                        text_color=sv.Color.black(),
                                        text_scale=0.5,
                                        text_thickness=1,
                                        text_padding=5)
        labels = [
            f"id:{tracker_id} {classes[class_id]} {confidence:0.2f}"
            for _, _, confidence, class_id, tracker_id in detections
        ]

        # # annotating bboxes
        # annotated_frame = box_annotator.annotate(
        #     scene=r.orig_img,  # scene=frame.copy()
        #     detections=detections,
        #     labels=labels
        # )
        annotated_frame = r.orig_img

        # append useful data to temp_table
        current_frame_list = []
        zone_frame_counter = [0 for _ in zones]
        for row in r.boxes.data.cpu().tolist():
            x1, y1, x2, y2, id, conf, class_no = row
            center = (round((x1+x2)/2, 2), round((y1+y2)/2, 2))
            bottom_center = (center[0], round(y2, 2))
            if CHECK_MODE == 'CENTER':
                pt_to_check = center
            elif CHECK_MODE == 'BOTTOM_CENTER':
                pt_to_check = bottom_center
            always_append_list = [int(id), int(class_no), conf, frame, x1, y1, x2, y2, center, bottom_center]

            zones_dependent_list = []
            for k, v in zones.items():
                is_in = Polygon(v).contains(Point(pt_to_check))
                zones_dependent_list.append(is_in)
                if is_in:
                    zones_objs_sets[k].add(int(id))

            # zones_dependent_list = [Polygon(z).contains(Point(pt_to_check)) for z in zones.values()]

            current_frame_list.append(always_append_list + zones_dependent_list)
            for i, zone in enumerate(zones_dependent_list):
                if zone == True:
                    zone_frame_counter[i] += 1

        for lista in current_frame_list:
            temp_table.append(lista + zone_frame_counter + [len(zones_objs_sets[k]) for k in zones_objs_sets.keys()]) # zone_total_counter

        # draw polygon zones on frame image
        color_gen = get_color()
        for k, v in zones.items():
            color = next(color_gen)
            cv2.polylines(annotated_frame, np.array([v]), isClosed=True, color=[c for c in reversed(color)], thickness=4)
        # fill zones with transparent color
        transparency = 77
        img = Image.fromarray(annotated_frame)
        drw = ImageDraw.Draw(img, 'RGBA')
        for k, zone in zones.items():
            drw.polygon(xy=zone, fill=(*[int(c+0.5) for c in reversed(zone_colors[k])], transparency))
        del drw
        annotated_frame = np.array(img)

        # draw zone annotations
        for i, zone_number in enumerate(zone_annotations.keys()):
            xy = zone_annotations[zone_number]
            x, y = xy[0], xy[1]
            text = f"z{zone_number} Now: {zone_frame_counter[i]} Total: {len(zones_objs_sets[zone_number])}"
            (text_width, text_height), baseline = cv2.getTextSize(text, tfontface, tscale, tthickness)
            cv2.rectangle(annotated_frame, xy, (x + text_width, y + text_height), (0,0,0), -1)
            cv2.rectangle(annotated_frame, xy, (x + text_width, y + text_height), (0,0,0), 20)
            cv2.putText(annotated_frame, text, (x, y+text_height), tfontface, tscale, [c for c in reversed(zone_colors[zone_number])], tthickness, tlinetype)
            
        # annotating bboxes
        annotated_frame = box_annotator.annotate(
            scene=annotated_frame,
            detections=detections,
            labels=labels
        )

        # write processed frame to video
        s.write_frame(annotated_frame)

# create df
df = pd.DataFrame(temp_table, columns=['id', 'class_no', 'confidence', 'frame', 'x1', 'y1', 'x2', 'y2', 'center', 'bottom_center'] + [f'zone{k}' for k in zones.keys()] + [f'zone{k}_FC' for k in zones.keys()] + [f'zone{k}_TC' for k in zones.keys()])

In [ ]:
pd.options.display.max_columns = 30
# pd.options.display.float_format = '{:.2f}'.format
pd.reset_option('float_format')
df  